# inception

![](https://norman3.github.io/papers/images/google_inception/f01.png)

![](https://miro.medium.com/max/2953/1*ZFPOSAted10TPd3hBQU8iQ.png)

In [3]:
!pip install torchinfo

In [4]:
import torch
import torch.nn as nn
from torchinfo import summary

In [5]:
# b 모형 구현.

# 각각의 모형
def conv_1(in_channels,out_channels):
    model=nn.Sequential(
    nn.Conv2d(in_channels,out_channels,kernel_size=1,padding=1,stride=1),
    nn.ReLU())
    return model

def conv_1_3(in_channels,mid_channels,out_channels):
    model=nn.Sequential(
    nn.Conv2d(in_channels,mid_channels,kernel_size=1,padding=1,stride=1),
    nn.ReLU(),
    nn.Conv2d(mid_channels,out_channels,kernel_size=3,padding=1,stride=1),
    nn.ReLU())
    return model

def conv_1_5(in_channels,mid_channels,out_channels):
    model=nn.Sequential(
    nn.Conv2d(in_channels,mid_channels,kernel_size=1,padding=1,stride=1),
    nn.ReLU(),
    nn.Conv2d(mid_channels,out_channels,kernel_size=5,padding=2,stride=1),
    nn.ReLU())
    return model

def max_3_1(in_channels,out_channels):
    model=nn.Sequential(
    nn.MaxPool2d(kernel_size=3,padding=1,stride=1),
    nn.Conv2d(in_channels,out_channels,kernel_size=1,padding=1,stride=1),
    nn.ReLU())
    return model


# inception -> 이것 또한 구성요소
class inception(nn.Module):
    def __init__(self,in_channels,out_channels_1,mid_channels_3,out_channels_3,mid_channels_5,out_channels_5,pool):
        super(inception,self).__init__()
        
        self.conv_1=conv_1(in_channels,out_channels_1)
        self.conv_1_3=conv_1_3(in_channels,mid_channels_3,out_channels_3)
        self.conv_1_5=conv_1_5(in_channels,mid_channels_5,out_channels_5)
        self.max_3_1=max_3_1(in_channels,pool)
    
    def forward(self,x):
        out_1=self.conv_1(x)
        out_2=self.conv_1_3(x)
        out_3=self.conv_1_5(x)
        out_4=self.max_3_1(x)
        output=torch.cat([out_1,out_2,out_3,out_4],dim=1) # N,C,H,W 의 C (0,1,2,3) 의 1
        return output
    
# 최종 GoogLeNet 같은 inception을 활용한 모델.
class GoogLeNet(nn.Module):
    def __init__(self,base_dim,num_classes=1000):
        super(GoogLeNet,self).__init__()
        
        self.layer_1=nn.Sequential(
        nn.Conv2d(3,base_dim,7,2,3), # in_channels,out_channels,kernel_size,stride,padding 순.
        nn.MaxPool2d(3,2,1), # kernel_size,stride,padding 순.
        nn.Conv2d(base_dim,base_dim*3,3,1,1), # 
        nn.MaxPool2d(3,2,1))
        
        self.layer_2=nn.Sequential(
        # in_channels,out_channels_1,mid_channels_3,out_channels_3,mid_channels_5,out_channels_5,pool
        inception(base_dim*3,64,96,128,16,32,32),
        inception(256,128,128,192,32,96,64),
        nn.MaxPool2d(3,2,1)
        )
        
        self.layer_3=nn.Sequential(
        inception(480,192,96,208,16,48,64),
        inception(512,160,112,224,24,64,64), # 알아보기 어렵다면, 0,1,2.. 라고 할때
        inception(512,128,128,256,24,64,64), # 1,3,5,6 만 더한게 in_channels로 간다.
        inception(512,112,144,288,32,64,64), # 바로 왼쪽을 예로 112+288+64+64=528
        inception(528,256,160,320,32,128,128),# 여기 528에서 256+320+128+128=832
        nn.MaxPool2d(3,2,1)
        )
        
        self.layer_4=nn.Sequential(
        inception(832,256,160,320,32,128,128),
        inception(832,384,192,384,48,128,128),
        nn.AvgPool2d(7,1) # kernel_size,stride 순.
        )
        
        self.layer_5=nn.Dropout2d(0.4) # p 확률
        self.fc_layer=nn.Linear(1024,num_classes)
        
    def forward(self,x):
        out=self.layer_1(x)
        out=self.layer_2(out)
        out=self.layer_3(out)
        out=self.layer_4(out)
        out=self.layer_5(out)
        # 원래 이게 맞는데
#       out=out.view(out.size(0),-1)
#       out=out.view(batch_size,-1) 이렇게 쓰여있었거든요. 근데 입출력 feature는 맞춰야하니까
# 아마 입력 H,W에서 틀린것같아요 마지막에 1,1024,1,1 이렇게 나와야 하는데 마지막에 1,1024,11,11 이래서 뭔가 이상한것같습니다.
        out=out.view(-1,1024)
        out=self.fc_layer(out)
        return out

In [6]:
model=GoogLeNet(3)
summary(model,(1,3,224,224))

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [1, 9, 28, 28]            --
|    └─Conv2d: 2-1                       [1, 3, 112, 112]          444
|    └─MaxPool2d: 2-2                    [1, 3, 56, 56]            --
|    └─Conv2d: 2-3                       [1, 9, 56, 56]            252
|    └─MaxPool2d: 2-4                    [1, 9, 28, 28]            --
├─Sequential: 1-2                        [1, 480, 16, 16]          --
|    └─inception: 2-5                    [1, 256, 30, 30]          --
|    |    └─Sequential: 3-1              [1, 64, 30, 30]           640
|    |    └─Sequential: 3-2              [1, 128, 30, 30]          111,680
|    |    └─Sequential: 3-3              [1, 32, 30, 30]           12,992
|    |    └─Sequential: 3-4              [1, 32, 30, 30]           320
|    └─inception: 2-6                    [1, 480, 32, 32]          --
|    |    └─Sequential: 3-5              [1, 128, 32, 32]          32,89

# resnet

![](https://neurohive.io/wp-content/uploads/2019/01/resnet-e1548261477164.png)

![](https://i.stack.imgur.com/kbiIG.png)

In [7]:
# bottleneck 블록을 위한. inception 같은거라고 보면 될거같다.

def conv_block_1(in_channels,out_channels,stride=1):
    model=nn.Sequential(
    nn.Conv2d(in_channels,out_channels,kernel_size=1,stride=stride,padding=1),
    nn.ReLU())
    return model

def conv_block_3(in_channels,out_channels):
    model=nn.Sequential(
    nn.Conv2d(in_channels,out_channels,kernel_size=3,stride=1,padding=1),
    nn.ReLU())
    return model

class BottleNeck(nn.Module):
    def __init__(self,in_channels,mid_channels,out_channels,down=False):
        super(BottleNeck,self).__init__()
        self.down=down
        
        if self.down:
            self.layer=nn.Sequential(
            conv_block_1(in_channels,mid_channels,stride=2),
            conv_block_3(mid_channels,mid_channels),
            conv_block_1(mid_channels,out_channels))
        
            self.downsample=nn.Conv2d(in_channels,out_channels,kernel_size=1,stride=2)
        else:
            self.layer=nn.Sequential(
            conv_block_1(in_channels,mid_channels),
            conv_block_3(mid_channels,mid_channels),
            conv_block_1(mid_channels,out_channels))
            
            self.dim_equalizer=nn.Conv2d(in_channels,out_channels,kernel_size=1)
            
    def forward(self,x):
        if self.down:
            downsample=self.downsample(x)
            out=self.layer(x)
            out=out+downsample
        
        else:
            out=self.layer(x)
            if x.size() is not out.size():
                x=self.dim_equalizer(x)
            
            out=out+x
        return out

In [8]:
class ResNet(nn.Module):
    def __init__(self,base_dim,num_classes=1000):
        super(ResNet,self).__init__()
        
        self.layer_1=nn.Sequential(
        nn.Conv2d(3,base_dim,7,2,3),
        nn.ReLU(),
        nn.MaxPool2d(3,2,1))
        
        self.layer_2=nn.Sequential(
        BottleNeck(base_dim,base_dim,base_dim*4),
        BottleNeck(base_dim*4,base_dim,base_dim*4),
        BottleNeck(base_dim*4,base_dim,base_dim*4,down=True))
        
        self.layer_3=nn.Sequential(
        BottleNeck(base_dim*4,base_dim*2,base_dim*8),
        BottleNeck(base_dim*8,base_dim*2,base_dim*8),
        BottleNeck(base_dim*8,base_dim*2,base_dim*8),
        BottleNeck(base_dim*8,base_dim*2,base_dim*8,down=True))
        
        self.layer_4=nn.Sequential(
        BottleNeck(base_dim*8,base_dim*4,base_dim*16),
        BottleNeck(base_dim*16,base_dim*4,base_dim*16),
        BottleNeck(base_dim*16,base_dim*4,base_dim*16),
        BottleNeck(base_dim*16,base_dim*4,base_dim*16),
        BottleNeck(base_dim*16,base_dim*4,base_dim*16),
        BottleNeck(base_dim*16,base_dim*4,base_dim*16,down=True))
        
        self.layer_5=nn.Sequential(
        BottleNeck(base_dim*16,base_dim*8,base_dim*32),
        BottleNeck(base_dim*32,base_dim*8,base_dim*32),
        BottleNeck(base_dim*32,base_dim*8,base_dim*32))
        
        self.avgpool=nn.AvgPool2d(7,1)
        self.fc_layer=nn.Linear(base_dim*32,num_classes)
        
    def forward(self,x):
        out=self.layer_1(x)
        out=self.layer_2(out)
        out=self.layer_3(out)
        out=self.layer_4(out)
        out=self.layer_5(out)
        out=self.avgpool(out)
        out=out.view(out.size(0),-1)
        out=self.fc_layer(out)
        return out

In [9]:
# 이건 image에 보통 많이 사용하나봐요 그래서 잔차학습을 했던 저번 반도체박막두께 대회 공부했던거하고
# 저희 main repo 의 issue를 보고 했는데 이건 x. 일단 뭔가 잘못된거같은데 모르겠습니다.
model=ResNet(3)
summary(model,(1,3,224,224))

RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: [Sequential: 1-1, Conv2d: 2-1, ReLU: 2-2, MaxPool2d: 2-3, Sequential: 3-1, Sequential: 4-1, Sequential: 4-2, Sequential: 4-3, Conv2d: 3-2]

In [10]:
model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=True)

Downloading: "https://github.com/pytorch/vision/archive/v0.6.0.zip" to /root/.cache/torch/hub/v0.6.0.zip
Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [11]:
# model=ResNet(3)
summary(model,(1,3,224,224))

Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [1, 64, 112, 112]         9,408
├─BatchNorm2d: 1-2                       [1, 64, 112, 112]         128
├─ReLU: 1-3                              [1, 64, 112, 112]         --
├─MaxPool2d: 1-4                         [1, 64, 56, 56]           --
├─Sequential: 1-5                        [1, 64, 56, 56]           --
|    └─BasicBlock: 2-1                   [1, 64, 56, 56]           --
|    |    └─Conv2d: 3-1                  [1, 64, 56, 56]           36,864
|    |    └─BatchNorm2d: 3-2             [1, 64, 56, 56]           128
|    |    └─ReLU: 3-3                    [1, 64, 56, 56]           --
|    |    └─Conv2d: 3-4                  [1, 64, 56, 56]           36,864
|    |    └─BatchNorm2d: 3-5             [1, 64, 56, 56]           128
|    |    └─ReLU: 3-6                    [1, 64, 56, 56]           --
|    └─BasicBlock: 2-2                   [1, 64, 56, 56]           --
|

## 저번 반도체 박막 두께 1등 resnet

![](https://ifh.cc/g/KdpCm5.jpg)

In [12]:
class ResNet(nn.Module):
    def __init__(self):
        super(ResNet, self).__init__()
        
        self.ln = nn.LayerNorm(10000)
        self.ln1 = nn.LayerNorm(7000)
        self.ln2 = nn.LayerNorm(4000)
        self.ln3 = nn.LayerNorm(2000)
        
        self.upblock1 = nn.Sequential(nn.Linear(226, 2000),nn.GELU(),nn.BatchNorm1d(2000))
        self.upblock2 = nn.Sequential(nn.Linear(2000,4000),nn.GELU(),nn.BatchNorm1d(4000))
        self.upblock3 = nn.Sequential(nn.Linear(4000,7000),nn.GELU(),nn.BatchNorm1d(7000))
        self.upblock4 = nn.Sequential(nn.Linear(7000,10000),nn.GELU(),nn.BatchNorm1d(10000))

        self.downblock1 = nn.Sequential(nn.Linear(10000, 7000),nn.GELU(),nn.BatchNorm1d(7000))
        self.downblock2 = nn.Sequential(nn.Linear(7000, 4000),nn.GELU(),nn.BatchNorm1d(4000))
        self.downblock3 = nn.Sequential(nn.Linear(4000, 2000),nn.GELU(),nn.BatchNorm1d(2000))
        self.downblock4 = nn.Sequential(nn.Linear(2000, 300),nn.GELU(),nn.BatchNorm1d(300))
        
        self.fclayer = nn.Sequential(nn.Linear(300,4))
        
    def forward(self, x):
        upblock1_out = self.upblock1(x)
        upblock2_out = self.upblock2(upblock1_out)
        upblock3_out = self.upblock3(upblock2_out)
        upblock4_out = self.upblock4(upblock3_out)
        
        downblock1_out = self.downblock1(self.ln(upblock4_out))
        skipblock1 = downblock1_out + upblock3_out
        downblock2_out = self.downblock2(self.ln1(skipblock1))
        skipblock2 = downblock2_out + upblock2_out
        downblock3_out = self.downblock3(self.ln2(skipblock2))
        skipblock3 = downblock3_out + upblock1_out
        downblock4_out = self.downblock4(self.ln3(skipblock3))
        
        output = self.fclayer(downblock4_out)
        
        return output

In [13]:
model=ResNet()
summary(model,(1,226))

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [1, 2000]                 --
|    └─Linear: 2-1                       [1, 2000]                 454,000
|    └─GELU: 2-2                         [1, 2000]                 --
|    └─BatchNorm1d: 2-3                  [1, 2000]                 4,000
├─Sequential: 1-2                        [1, 4000]                 --
|    └─Linear: 2-4                       [1, 4000]                 8,004,000
|    └─GELU: 2-5                         [1, 4000]                 --
|    └─BatchNorm1d: 2-6                  [1, 4000]                 8,000
├─Sequential: 1-3                        [1, 7000]                 --
|    └─Linear: 2-7                       [1, 7000]                 28,007,000
|    └─GELU: 2-8                         [1, 7000]                 --
|    └─BatchNorm1d: 2-9                  [1, 7000]                 14,000
├─Sequential: 1-4                        [1, 10000]    